In [1]:
SEED = 42

import random
import numpy as np
import skmultilearn
import pandas as pd
from sklearn.metrics import classification_report, f1_score
import numpy as np
from sklearn import model_selection

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

import spacy
# !pip install --user spacy
# !python3 -m spacy download pt_core_news_sm

In [3]:
test_set = pd.read_csv('../sets/lemmatized Set 0.csv', index_col=0)
set_1 = pd.read_csv('../sets/lemmatized Set 1.csv', index_col=0)
set_2 = pd.read_csv('../sets/lemmatized Set 2.csv', index_col=0)
set_3 = pd.read_csv('../sets/lemmatized Set 3.csv', index_col=0)
set_4 = pd.read_csv('../sets/lemmatized Set 4.csv', index_col=0)


def string_to_array_serie(serie):
    return [[int(i) for i in t.replace("[", "").replace("]", "").split()] for t in serie]

test_set["Senado"] = string_to_array_serie(test_set["Senado"])
set_1["Senado"] = string_to_array_serie(set_1["Senado"])
set_2["Senado"] = string_to_array_serie(set_2["Senado"])
set_3["Senado"] = string_to_array_serie(set_3["Senado"])
set_4["Senado"] = string_to_array_serie(set_4["Senado"])

test_set["Referenda"] = string_to_array_serie(test_set["Referenda"])
set_1["Referenda"] = string_to_array_serie(set_1["Referenda"])
set_2["Referenda"] = string_to_array_serie(set_2["Referenda"])
set_3["Referenda"] = string_to_array_serie(set_3["Referenda"])
set_4["Referenda"] = string_to_array_serie(set_4["Referenda"])

In [4]:
train_sets = [set_1, set_2, set_3, set_4]

full_train = pd.concat(train_sets)
full_train

,Numero ato,Ementa + texto completo,Senado,Referenda
0,10095,dispor sobrar o comitê consultivo de nanotecno...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
1,10092,promulgar o protocolo de integração educativo ...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10085,dispor sobrar o programar forçar o esporte seg...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0]"
3,10081,alterar o decretar n 8713 de 15 de abril de 20...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]"
4,10083,autorizar o empregar das forçar armar o garant...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
1323,3336,dar novo redação aos arts 11 15 16 19 e 30 d...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
1324,3330,dispor sobrar o redução do consumir de energia...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1325,3338,aprovar o estruturar regimental e o quadrar de...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]"
1326,3328,alterar o decretar 2889 de 21 12 1998 que disp...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"


In [5]:
Train_X = full_train["Ementa + texto completo"]
Test_X = test_set["Ementa + texto completo"]
set_1_X = set_1["Ementa + texto completo"]
set_2_X = set_2["Ementa + texto completo"]
set_3_X = set_3["Ementa + texto completo"]
set_4_X = set_4["Ementa + texto completo"]

### Generating stopwords table with term frequencies

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def removeStopWords(texts, st= stopwords.words('portuguese')):
    texts_r = []
    for t in texts:
        final_t = ""
        for w in t.split(" "):
            if w not in st and w!=".":
                final_t += w + " "
        texts_r.append(final_t)
    return texts_r

Train_X = removeStopWords(Train_X)
Test_X = removeStopWords(Test_X)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/caiocampos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
contagem = dict()
for doc in Train_X:
    for word in doc.split(" "):
        if word in contagem.keys():
            contagem[word]+=1
        else:
            contagem[word]=1
            

contagem_interdoc = dict()
for doc in Train_X:
    for word in list(set(doc.split(" "))):
        if word in contagem_interdoc.keys():
            contagem_interdoc[word]+=1
        else:
            contagem_interdoc[word]=1
            
word_frequency = pd.DataFrame(columns=["Word", "Qtd_total", "Qtd_interdocumental","stopword"])
i=0
for word, number in sorted(contagem.items(), key=lambda x: x[1], reverse=True):
    word_frequency.loc[i] = {"Word": word, "Qtd_total": number, "Qtd_interdocumental":contagem_interdoc[word], "stopword": 0}
    i+=1

word_frequency

,Word,Qtd_total,Qtd_interdocumental,stopword
0,,289627,5294,0
1,art,63400,5294,0
2,decretar,41258,5294,0
3,n,37251,5112,0
4,ser,35473,3905,0
...,...,...,...,...
24350,99656,1,1,0
24351,1888,1,1,0
24352,3154,1,1,0
24353,3222,1,1,0


In [9]:
word_frequency.to_csv("stopwords.csv")

### Generating list of stopwrods from manually analysed file
This procedure pressuposes that the manual selection of stopwords was already performed.

In [20]:
stopwords = pd.read_csv('stopwords.csv')
stopwords

,Unnamed: 0,Word,Qtd_total,Qtd_interdocumental,stopword
0,0,NaN,289627,5294,0
1,1,art,63400,5294,1
2,2,decretar,41258,5294,1
3,3,n,37251,5112,1
4,4,ser,35473,3905,1
...,...,...,...,...,...
24350,24350,99656,1,1,0
24351,24351,1888,1,1,0
24352,24352,3154,1,1,0
24353,24353,3222,1,1,0


In [21]:
frequency_stopwords_list = []
for sw in stopwords.values:
    if sw[4] == 1:
        frequency_stopwords_list.append(sw[1])
pd.DataFrame(frequency_stopwords_list).to_csv("frequency_stopwords_list.csv")